# Road Following Data Editing - Adding xy Coordinates to Photos by Mouseclick

Author: Travis Moore, NNMC STARR Team

Note: This code draws heavily from three sources. First, the NVIDIA data_collection.ipynb notebook shipped with the Jetbot sold by Sparkfun.com. You can find the original code here: https://github.com/NVIDIA-AI-IOT/jetbot

Second, George Gorospe's modified data collection notebook that uses the Jupyter Clickable Widgets library to select a best path by mouseclick rather than scrollbars (https://github.com/jaybdub/jupyter_clickable_image_widget). 

Third, the BalticBot Poland STARR team's fantastic notebook that provides functionality to mark X,Y coordinates on existing images.

In [ ]:
# Import libraries
# IPython Libraries for display and widgets
import ipywidgets.widgets as widgets
from IPython.display import display
from ipywidgets import interact, interact_manual
from jupyter_clickable_image_widget import ClickableImageWidget

# Python basic packages for image annotation
import os
import numpy as np
import cv2
import time

# Create a list of photos from an existing directory
DIR = 'dataset1'
photo_list = os.listdir(DIR)
nr_photos = len(photo_list)
print("You have %d photos in that folder." % nr_photos)

# Select the first photo from the list
file = photo_list[0]
print(file)

# Encode the first file as an image
img = cv2.imread(DIR+'/'+file)
imv=cv2.imencode('.jpg', img)[1].tobytes()

# Display the first image
image_widget = widgets.Image(value = imv, format='jpeg', width=224, height=224)
display(image_widget)

# Create an image to identify the end of the dataset
end_img = np.zeros((224,224,3), np.uint8)
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(end_img,'END OF DATASET',(60,120), font, 0.4,(255,255,255),1,cv2.LINE_AA)
imv_end=cv2.imencode('.jpg', end_img)[1].tobytes()
image_widget.value = imv_end
display(image_widget)

# Create a new directory to save the modified images
DATASET_XY_DIR = 'new_dataset_xy'
try:
    os.makedirs(DATASET_XY_DIR)
except FileExistsError:
    print('Directory not created because it already exists')

# Create save button
button_layout = widgets.Layout(width='128px', height='64px', align_self='center')
saveButton = widgets.Button(description='SAVE', button_style='success', layout=button_layout)

# Apply the x and y offset data to the current image's title and save in the new directory
def save_photo(_, content, msg):
    global count, xs, ys

    file = photo_list[count]
    
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
    
        img = cv2.imread(DIR + '/' + file)
        imv = cv2.imencode('.jpg', img)[1].tobytes()
        new_file = 'xy_%03d_%03d_%s' % (xs * 50 + 50, ys * 50 + 50, file)
        image_path = os.path.join(DATASET_XY_DIR, new_file)
            with open(image_path, 'wb') as f: 
                f.write(imv)

# Function to save the current image and move on to the next one
def next_photo():
    global count, xd, yd, xs, ys
    save_photo()
    count = (count + 1)
    if count < len(photo_list):
        file = photo_list[count]
        img = cv2.imread(DIR + '/' + file)
        if content['event'] == 'click':
            data = content['eventData']
            x = data['offsetX']
            y = data['offsetY']
            img = cv2.circle(img, (xd,yd), 8 (0,255,0), 3)
            imv = cv2.imencode('.jpg', img)[1].tobytes()
            image_widget.value = imv
    else:
        image_widget.value = imv_end

# Assign function to save button
saveButton.on_click(lambda x: next_photo())

count = 0
@interact
def show_images():
    global count, xd, yd, xs, ys
    if count < len(photo_list):
        file = photo_list[count]
        img = cv2.imread(DIR + '/' + file)
        image = np.copy(img)
        xs = x
        ys = y
        if content['event'] == 'click':
            data = content['eventData']
            x = data['offsetX']
            y = data['offsetY']
        image = cv2.circle(image, (x,y), 8, (0,255,0), 3)
        imv = cv2.imencode('.jpg', image)[1].tobytes()
        image_widget.value = imv
        xd = x
        yd = y

    display(widgets.Hbox([image_widget, saveButton]))